# App Advertisement User Analysis 

In this mini-project, I am analyzing products on the app store markets to understand what kind of apps are likely to attract the most users. As a data analyst that works for a company that builds free to download apps, it is my job to enable the engineering and product teams to make data-driven decisions for the apps they are planning to build.   

The company I contracted to work for only builds free to download apps, so their only source of revenue is in-app advertisements. This means that the revenue of our apps is directly proportional to how many users see the advertisements. The goal of this project is to find trends and insights about apps that attract users. 


## Collecting Data

As of 2020, 2 million iOS apps exist on the App Store, and 2.1 million Android apps exist on the Google Play store.
Collecting data for 4 million apps is going to take a tremendous amount of time and resources, so I will analyze a sample of the data instead. Luckily, two data sets on Kaggle exist that are suitable for my goal. 

[A dataset](https://www.kaggle.com/datasets/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google.

[A dataset](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store.

Let's begin by opening and reading the two datasets.

In [3]:
from csv import reader

opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
iOS_data = list(read_file)

iOS_header = iOS_data[0]
iOS_data = iOS_data[1:]


opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android_data = list(read_file)

android_header = android_data[0]
android_data = android_data[1:]

To help us explore the datasets, I will write a function named `explore_data()` to retrieve a selection of rows in a more readable format. Another feature included in the function is showing the number of rows and columns in the dataset.

In [4]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        

In [5]:
print(iOS_header)
explore_data(iOS_data, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


As we can see from the output of our function, the iOS dataset contains 7197 rows and 16 columns of categories. Several categories that grab my attention are `track_name`, `price`, `user_rating`, `reviews`, `rating_count_tot`, and `prime_genre`.

In [6]:
print(android_header)
explore_data(android_data, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


As we can see from the output of our function, the Android dataset contains 10841 rows and 13 columns of categories. Several categories that grab my attention are `Category`, `Size`, `Type`, `Installs`, `Content Rating`, `Genres`, and more.

## Cleaning the Data

Before beginning my analysis, I need to make sure that the data provided is accurate and lean, or else the results of my analysis will be erroreous.

This means I need to detect inaccurate data, remove, or change it. It's often said that data analysts spend about 75% of the time cleaning data and only 25% actually analyzing data. In this project, much of my time is spent cleaning the two data sets.


### Deleting Bad Data

The Google Play dataset has a dedicated [discussion section](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion), and we can see that [one of the discussions](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015) describes an error for row 10472. Let's explore this row.

In [7]:
print(android_header)
print(android_data[10472])

print(android_data[1])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Row 10472 corresponds to the app information for *Life Made Wi-Fi Touchscreen Photo Frame*. The rating for this app is 19, which can't be correct because 5 is the max rating value. As documented in the link above, this error is caused by the absence of the correct `Category` value. As a result, I will delete this row. 

In [8]:
print(len(android_data))
del(android_data[10472])
print(len(android_data))


10841
10840


### Deleting Duplicate Entries

Another process of data cleaning is deleting duplicate entries. If you sift through Google Play's dataset for long enough, you'll notice some apps have duplicate entries. For example, Instagram has four entries.

In [9]:
for app in android_data: 
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


We can further explore this by writing a function to filter the number of duplicate apps that exist in this dataset. 

In [10]:
duplicate_apps = []
unique_apps = []

for app in android_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('The number of duplicate apps:', len(duplicate_apps))
print('The number of unique apps:', len(unique_apps))

print('Examples of duplicate apps:', duplicate_apps[:10])


The number of duplicate apps: 1181
The number of unique apps: 9659
Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


As visualized above, over 1180 apps are duplicated throughout the dataset.   
We don't want to count certain apps more than once when we analyze data, so let's remove the duplicates and keep only one entry per app. 
WE could remove the duplicate rows randomly, but let's find a better way.

In [11]:
for app in android_data: 
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


The main difference found in the four varying rows for Instagram is column 4, which represents the number of reviews. It seems like Instagram was collected as data at several different times, because the number of reviews gets larger. 
We can use this information to build a criterion for removing duplicates. The larger the number, the more recent the data entry for Instagram should. Rather than removing the duplicates randomly, I will keep the row with the highest number of reviews and remove the other entries. 

One way to handle this technical challenge is using a dictionary, where the key is the unique app name and the value is the updated with the highest number of reviews of that app.

In [12]:
reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
        
print('expected length:', len(android_data) - len(duplicate_apps))
print('actual length:', len(reviews_max))

expected length: 9659
actual length: 9659


Perfect, we have done exactly what we wanted to do, which was to create a data structure containing only one copy of the apps in the original Android dataset. Both lengths match, so everything went as expected. The length of my new dictionary is the same length as the Android dataset with the duplicates subtracted. 

Next, I will use the dictionary, `reviews_max`, as a method of comparison to remove any duplicates with smaller number of reviews. 

We will only keep the duplicate entry with the highest number of reviews in the newly initalized list, `android_clean`. In the code below,
* I initalized two new empty lists `android_clean` and `already_added`.
* Next, I iterated over the original android dataset and extracted the name and number of reviews and stored the information in their respective variables.
* If the number of reviews in the current app equals the value stored in the key that matches the name in the reviews_max dictionary AND the name is not already found in the new dictionary already_added, then append the app and name to their respective new lists. 

In [13]:
android_clean = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if(n_reviews == reviews_max[name]) and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

print(len(android_clean))

9659


`android_clean` is exactly 9659 units long, which is the same length as the `reviews_max` dictionary we created earlier. This confirms that our code works. 

Now, we just looked at Android's dataset and cleaned it a little. It's time to look at Apple's app store data.  

In [14]:
duplicate_apps = []
unique_apps = []

for app in iOS_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('The number of duplicate apps:', len(duplicate_apps))
print('The number of unique apps:', len(unique_apps))

The number of duplicate apps: 0
The number of unique apps: 7197


Using the code to count for duplicates again for the iOS dataset, it seems that there are 0 duplicates in this dataset, which saves us a lot of work. 

### Cleaning Non-English Language Apps

In our analysis, we are only concerned about apps for English-speaking audiences. This means we need to remove any apps in our dataset that contains symbols not commonly used in English. 

To solve this challenge, we can use our understanding of the ASCII system, which states that all English text symbols correspond to numbers between 0 and 127. If an app name contains a character that is greater than 127, it is likely that the app has a non-English name. We can use the built-in function `ord()` to help us identify these data points. 

In [15]:
def isEnglish(string):
    for letter in string:
        num_char = ord(letter)
        if num_char > 127:
            return False 
    return True

print(isEnglish('Instagram'))
print(isEnglish('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(isEnglish('Docs To Go™ Free Office Suite'))
print(isEnglish('Instachat 😜'))

True
False
False
False


However, there are several edge cases that break this line of thinking expressed in code above. 😜 and the ™ sign are common enough to be considered part of the English language, but their equivalent ASCII value have corresponding values over 127, so my function deleted some false negatives. We're going to lose useful data because some English apps will be labeled incorrectly as non-English. To minimize the data loss, we will remove an app only if its name has more than three characters that correspond outside the ASCII range. 

In [16]:
def isEnglish(string):
    odd_char_count = 0
    for letter in string:
        num_char = ord(letter)
        if num_char > 127:
            odd_char_count += 1 
    if odd_char_count > 3:
        return False 
    else: 
        return True

print(isEnglish('Docs To Go™ Free Office Suite'))
print(isEnglish('Instachat 😜'))
print(isEnglish('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


This is much better. The app titled `Instachat 😜` is now considered an English app. 

In [17]:
android_english = [] 
ios_english = []

for app in android_clean:
    name = app[0]
    if isEnglish(name):
        android_english.append(app)
    
    
for app in iOS_data:
    name = app[1]
    if isEnglish(name):
        ios_english.append(app)
        
explore_data(android_english,0,3,True)
print('\n')
explore_data(ios_english,0,3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

The android dataset downsized from 9659 to 9614 rows. The iOS dataset downsized from 7197 to 6183 rows. 

### Cleaning Non-Free Apps 

In this data cleaning process, we're done the following:
* Removing inaccurate data
* Removing duplicate data
* Removing non-English apps

As established in the beginning, I work for a company that only builds free apps. Therefore, one of the most critical steps of data cleaning is to remove any apps that are non-free. I will write a function to help me filter for free apps, which will be our last step in data cleaning. 

In [18]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == "0":
        android_final.append(app)
    
for app in ios_english:
    price = app[4]
    if price == "0.0":
        ios_final.append(app)
        
print("Number of free android apps:", len(android_final))
print("Number of free iOS apps:", len(ios_final))

Number of free android apps: 8864
Number of free iOS apps: 3222


## Analyzing the Data 

Now that we have clean data, we can finally begin our analysis.

Because our end goal is to add the free app on both Google Play and the App Store, we will need to find app profiles that are successful in both markets. 

To minimize risk and overhead, our validation strategy for an app idea has three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six month,s we build an iOS version of the app and add it to the App Store.

Let's begin our analysis by determining the most common genres for each market. For this, we will need to build a frequency table for a few columns in our datasets and identify the columns which might be useful for determining the most common genres.

In [19]:
def freq_table(dataset, index):
    freq_table = {}
    total = 0
    
    for app in dataset:
        total += 1
        category = app[index]
        if category in freq_table:
            freq_table[category] += 1
        else:
            freq_table[category] = 1
    
    percentage_table = {}
    
    for key in freq_table:
        percentage_table[key] = 100 * (freq_table[key] / total)
        
    return percentage_table

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [20]:
display_table(ios_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The most common genre is Games with a staggering 58%, followed by Entertainment with 8%. 
I see a pattern of leisure and entertainment apps categories being more popular than the more practical genres of apps. 
Making a gaming app would be a strategic decision because over 50% of the app market is games. I wouldn't be surprised if there was a correlation between number of users and the popularity of video games on the market.

In [25]:
display_table(android_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [26]:
display_table(android_final, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

As we can see here from our data in the Google Play dataset, a different pattern emerges from the genres ranking. Tools leads with a 8.4% share of the market. Practical genres are more popular on the Google Play dataset, which deviates from the trend observed in the App Store market. A recommendation for an app to be popular on both markets would be one that combines practicality with gaming or entertainment. It is inconclusive to assume that the most frequent app genres have the most users. Further analysis needs to be explored to reveal which type of apps have the most users. 

The most popular category, as shown in the first query, reveals that the Family category leads with a noteworth 19% of the market percentage. A quick google search will show that Family is actually the kids friendly category. Once again, games is found at the top of the rankings with a 10% market percentage. Many Family-categorized apps are actually games as well, so these categories are loosely defined. 

### Finding Users Per Genre 

One way to find which genres have the most users is to calculate the average number of installs of each app genre. For the Google Play dataset, we can find this information in the Installs column, but this information is missing in the App Store dataset. As a workaround, we'll take the total number of user ratings found in `rating_count_tot`. 

In [40]:
genre_table = freq_table(ios_final, -5)

for genre in genre_table:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:
            num_users = float(app[5])
            total += num_users
            len_genre += 1
    avg_users = total / len_genre
    print(str(genre) +' : '+ str(avg_users))
    

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Navigation, social networking, reference, and education are among the most popular genres. Navigation has the most number of average viewers with a total of 86

In [51]:
category_freq = freq_table(android_final, 1)

for category in category_freq:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:
            num_installs = app[5]
            num_installs = num_installs.replace("+", "")
            num_installs = num_installs.replace(",", "")
            total += float(num_installs)
            len_category += 1
    avg = total / len_category
    print(category + " : " + str(avg))

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On average, communcications have the most installs, with an average of 38 million average installs per app. However, this data point is heavily distorted by outliers like monoliths like Facebook Messenger, Google Chrome, WhatsApp, etc. with over billions of downloads each. 

In [52]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps above that have over 100 million installs, the average would be reduced roughly ten times:



In [57]:
under_100m = []

for app in android_final:
    num_installs = app[5]
    num_installs = num_installs.replace("+", '')
    num_installs = num_installs.replace(",", '')
    if app[1] == "COMMUNICATION" and float(num_installs) < 100000000:
        under_100m.append(float(num_installs))

print("avg without outliers: " + str(sum(under_100m) / len(under_100m)))

avg without outliers: 3603485.3884615386


Without the apps in the list above inflating the average, our new average for communications installs is 3.5million.

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against (Google, Amazon, Facebook). 

## Conclusions


In this project, we cleaned and analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

However, there is always room for a great idea that is executed well. Entertainment and games rank highly in both markets. Since the only revenue source for our app will be in-app advertisements, I recommend we build a simple yet highly rewarding video game for all audiences (youth-friendly). It wouldn't make sense for us to target a market segment with moderately low average downloads, as the audience for such markets are naturally smaller. As we discovered, categories and genres are dominated by several outliers, which tremendously skew the averages of installs per category. Finding an untapped niche for a highly successful app profile will require greater analysis. This project was simply an exercise to practice analysis without a Python library like Pandas. 